In [1]:
#Initial Example Code is Pasted Below

import requests



# Define the headers to send to the API URI
headers = {
    'content-type':'application/x-www-form-urlencoded;',
    'Access-Control-Allow-Origin':'*',
#    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:27.0) Gecko/20100101 Firefox/27.
}

#Compound can be a list of strings, be sure to use empirical formula i.e. "Au", not "Gold"
compound = ["Cu"]
#x-axis variable, use code for in decodre table, not name of the variable i.e. Temperature: "T"
var = "T"
#property attempting to measure, use code in decoder table, not name i.e. Electrical Resisitivity: "ER"
prop = "ER"
#representation of data, use code in decoder table, i.e. Direct: "A", if no representation wanted, use bool False
rep = False
#state of matter for data, use code in decoder table, i.e. Crystal: "C", if all states wanted, use bool False
state = False
#whether or not you want to show error bars for points on the graph, True to show, False to hide
show_uncertainty = True

# Define the search data JSON to send to the API URI
search_data = {"property_search_code":prop}
# search_data = {"author_last_name":['Wilthan']}

#"exclude_all":"True", "required_compounds":compound, "property_search_code":"VDN"

# Define the URL to send the request
# Note that this example does require an authentication key to get the data. The one given here is an invalid authkey and will need to be replaced by
# a valid authentication key which may be requested free of charge fapi_key = open('C:/Users/bmd4/Desktop/API Work/TRC-Alloys API Key.txt').read()r# om TRCalloy@nist.gov.

api_key = open('D:/Bryan/Desktop/NIST/TRC-Alloys API Key.txt').read()
# api_key = open('C:/Users/bmd4/Desktop/API Work/TRC-Alloys API Key.txt').read()
url = f'http://trcsrv2.boulder.nist.gov/MetalsAlloyAPI/search?authkey={api_key}'
# url=f'https://trc.nist.gov/MetalsAlloyAPI/search?authkey={api_key}'

In [2]:
import re
# Get the compound data from a requests post
compound_data_response = requests.post(url, json=search_data, headers=headers)
# The compound_data object is a "response" object and the actual output JSON is in the
# text field of that object. So use compound_data_response.text to use the actual output JSON
with open('Test.json', 'w') as fp:
    fp.write((re.sub(r'[^\x00-\x7f]',r'', compound_data_response.text)))

In [3]:
import json

with open('Test.json') as json_file:
    data = json.load(json_file)

with open('decoder_table.json') as json_file:
    decoder_table = json.load(json_file)

In [4]:
from collections import Counter
element_ids = []
for a in data['TRC_data']:
    for b in a['compounds']:
        for c in compound:
            if b['formula_hill'] == c:
                c = [b['compound_id']]
                if c[0] not in element_ids:
                    element_ids.append(c[0])
print(element_ids)

[7671]


In [5]:
x_lab = decoder_table['PRP'][var]
y_lab = decoder_table['PRP'][prop]

In [6]:
import pandas as pd
import itertools
temp_x = "N/A"
temp_y = "N/A"
state_dict = {}
dataset = []
num1 = 0
num2 = 0
num3 = 0
num4 = 0
num5 = 0

for a in data['TRC_data']:
    for b in a['systems']:
        try:
            if (Counter(b['compound_ids']) == Counter(element_ids)) or (compound == False):
                num3 += 1
                for p in b['phases']:
                    state_dict[p['phase_id']] = p['type']
                for c in b['data_sets']:
                    num5 += 1
                    x_axt = []
                    y_axt = []
                    stat_id = []
                    y_unc = []
                    add_data = False
                    method = ""
                    for s in c['states']:
                        stat_id.append(state_dict[s['phase_id']])
                    for v in c['variables']:
                        if v['variable_name'] == prop:
                            num1 += 1
                            temp_y = v['variable_id']
                            rep_u = v['representation']
                            if f"'{v['units']}':" in str(decoder_table['UnitsSpecial']):
                                y_un = decoder_table['UnitsSpecial'][v['units']]
                            else:
                                y_un = v['units']
                            if 'method' in str(v):
                                if str(v['method']).upper() in str(decoder_table['Method']):
                                    method = decoder_table['Method'][v['method'].upper()]
                                else:
                                    method = v['method']
                            else:
                                method = ""
                                print(f"NO METHOD LISTED: {c['data_set_id']}")
                        if v['variable_name'] == var:
                            num2 += 1
                            temp_x = v['variable_id']
                            if f"'{v['units']}':" in str(decoder_table['UnitsSpecial']):
                                x_un = decoder_table['UnitsSpecial'][v['units']]
                            else:
                                x_un = v['units']
                    for d in c['data']: # goes through data tables and eventially data values
                        if (f"'variable_id': {temp_x}" in str(c['data'])) and (f"'variable_id': {temp_y}" in str(c['data'])): # only enters if both property and variable are compatible
                            if d['variable_id'] == temp_y:
                                for e in d['data_values']:
                                    if e['value'] == None:
                                        y_axt.append(None)
                                        y_unc.append(None)
                                    else:
                                        if "'uncertainty':" in str(e): # adds the uncertainty value for use in the data framework
                                            y_axt.append(e['value'])
                                            y_unc.append(e['uncertainty'])
                                        else:
                                            y_axt.append(e['value'])
                                            y_unc.append(None)
                                            if (type(e['value']) == int or type(e['value']) == float):
                                                print(f"NO UNCERTAINTY LISTED: {c['data_set_id']}")
                            if d['variable_id'] == temp_x:
                                for f in d['data_values']:
                                    x_axt.append(f['value'])
                            add_data = True
                            num4 += 1
                    if add_data == True:
                        for i,j,k in itertools.zip_longest(x_axt,y_axt,y_unc):
                            if i == None:
                                i = x_axt[0]
                            dataset.append({
                                f'{x_lab}: {x_un}': i,
                                f'{y_lab}: {y_un}': j,
                                'Uncertainty': k,
                                'Data Set ID': str(c['data_set_id']),
                                'State': stat_id,
                                'Representation': rep_u,
                                'Method': method,
                                'Year': a['citation']['year']
                            })
        except KeyError:
            print(a['citation']['citation_id'])

print(num3, num1, num2, num4, num5)
df = pd.DataFrame(dataset)

6386
6899
6899
6899
6899
6899
6899
6899
7122
7122
7122
7122
7122
7122
7122
9804
80 140 199 360 212


In [7]:
import plotly.express as px
import math

x = []
y = []

n = 0
for a, b, c in zip(df['Representation'], df['State'], df[f'{y_lab}: {y_un}']):
    if (a != rep) and (rep != False):
        df = df.drop(df.index[n], axis = 0)
        n -= 1
    elif (b != [state]) and (state != False):
        df = df.drop(df.index[n], axis = 0)
        n -= 1
    elif math.isnan(c):
        df = df.drop(df.index[n], axis = 0)
        n -= 1
    n += 1

if show_uncertainty:
    fig = px.scatter(df, x=f'{x_lab}: {x_un}',y=f'{y_lab}: {y_un}', title=str(compound), color='Data Set ID', hover_data=[df.State, df.Representation, df.Method, df.Year], error_y='Uncertainty', height=600)
else:
    fig = px.scatter(df, x=f'{x_lab}: {x_un}',y=f'{y_lab}: {y_un}', title=str(compound), color='Data Set ID', hover_data=[df.State, df.Representation, df.Method, df.Year], height=600)

fig.show()

# pd.set_option('display.max_rows', None)
df

,Temperature: kg/cm**2,Electrical resistivity: ohm*m,Uncertainty,Data Set ID,State,Representation,Method,Year
18,96.35,3.750000e-09,NaN,451,[C],A,DC potentiometric method,1908
19,97.45,3.750000e-09,NaN,451,[C],A,DC potentiometric method,1908
20,99.25,3.840000e-09,NaN,451,[C],A,DC potentiometric method,1908
21,100.15,3.940000e-09,NaN,451,[C],A,DC potentiometric method,1908
22,100.45,3.980000e-09,NaN,451,[C],A,DC potentiometric method,1908
...,...,...,...,...,...,...,...,...
1197,1836.00,2.537800e-07,1.091250e-08,26460,[L],A,Resistive pulse heating,2012
1198,1876.00,2.569100e-07,1.104710e-08,26460,[L],A,Resistive pulse heating,2012
1199,1916.00,2.600400e-07,1.118170e-08,26460,[L],A,Resistive pulse heating,2012
1200,1956.00,2.631700e-07,1.131630e-08,26460,[L],A,Resistive pulse heating,2012
